In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!unzip -qq "gdrive/My Drive/research/train.zip"
!unzip -qq "gdrive/My Drive/research/test.zip"

In [3]:
import torch
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

1
Tesla P100-PCIE-16GB


In [4]:
import os
import pandas as pd
import torch
#from sklearn import preprocessing

from PIL import Image
from torch.utils.data import Dataset

class AksaraBali(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file, header=None)
        self.root_dir = root_dir
        self.transform = transform
       
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index,0])
        image = Image.open(img_path).resize((224, 224))
        y_label = torch.tensor(self.annotations.iloc[index,2], dtype=torch.long)
        
        if self.transform:
            image = self.transform(image)
            
        return(image, y_label)

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def check_accuracy(loader, model, acc_type):
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device='cuda:0')
            y = y.to(device='cuda:0')
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        
        print(f'{acc_type} got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}') 
    
    model.train()
    
in_channel = 3 
num_classes = 133
learning_rate = 0.001
batch_size = 64
num_epoch = 250

# tranform
my_transform = transforms.Compose([transforms.ToTensor(),
                                   transforms.Normalize(mean=(0.5005, 0.4670, 0.4174), std=(0.0813, 0.0534, 0.0932))])

train_set = AksaraBali(csv_file="gdrive/My Drive/research/train_label.csv",
                       root_dir="train",
                       transform=my_transform)

test_set = AksaraBali(csv_file="gdrive/My Drive/research/test_label.csv",
                       root_dir="test_image_random",
                       transform=my_transform)

#train_set = torch.utils.data(dataset)
train_loader = DataLoader(dataset=train_set,
                          batch_size=batch_size,
                          shuffle=True)

test_loader = DataLoader(dataset=test_set,
                          shuffle=False,
                         batch_size=batch_size)
#test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

model = torchvision.models.resnet50(pretrained=True)
model.to('cuda:0')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epoch):
    losses = []
    
    for batch_idx, (data, targets) in enumerate(train_loader):
        # get data to cuda
        data = data.to(device='cuda:0')
        targets = targets.to(device='cuda:0')
        
        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        
        losses.append(loss.item())
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        
        # gradient descent or adam step
        optimizer.step()
    
    print(f'Cost at epoch {epoch+1} is {sum(losses)/len(losses)}')
    check_accuracy(train_loader, model, "Train accuracy ")
    check_accuracy(test_loader, model, "Test accuracy ")
   
#torch.save(model, "model1810.pt")

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


## RESNET18 64batch 

Cost at epoch 0 is 0.9688131187935002
Got 6736 / 7673 with accuracy 87.79
Cost at epoch 1 is 0.19390780546774083
Got 6714 / 7673 with accuracy 87.50
Cost at epoch 2 is 0.11893820669725895
Got 6643 / 7673 with accuracy 86.58
Cost at epoch 3 is 0.08210397435155195
Got 6742 / 7673 with accuracy 87.87
Cost at epoch 4 is 0.07016220813182791
Got 6701 / 7673 with accuracy 87.33
Cost at epoch 5 is 0.07725537565308443
Got 6755 / 7673 with accuracy 88.04
Cost at epoch 6 is 0.04713408879167848
Got 6872 / 7673 with accuracy 89.56
Cost at epoch 7 is 0.03732098474770632
Got 6632 / 7673 with accuracy 86.43
Cost at epoch 8 is 0.05611207986378129
Got 6836 / 7673 with accuracy 89.09
Cost at epoch 9 is 0.03829338655626858
Got 6806 / 7673 with accuracy 88.70
Cost at epoch 10 is 0.02294694669371193
Got 6709 / 7673 with accuracy 87.44
Cost at epoch 11 is 0.04540037233276122
Got 6868 / 7673 with accuracy 89.51
Cost at epoch 12 is 0.025756649938837375
Got 6775 / 7673 with accuracy 88.30
Cost at epoch 13 is 0.045317758210260836
Got 6873 / 7673 with accuracy 89.57
Cost at epoch 14 is 0.025782721148901216
Got 6997 / 7673 with accuracy 91.19
Cost at epoch 15 is 0.019851571001219363
Got 6988 / 7673 with accuracy 91.07
Cost at epoch 16 is 0.03279321799664103
Got 6806 / 7673 with accuracy 88.70
Cost at epoch 17 is 0.025503783545655797
Got 6927 / 7673 with accuracy 90.28
Cost at epoch 18 is 0.018462478041048877
Got 6820 / 7673 with accuracy 88.88
Cost at epoch 19 is 0.015971610776659872
Got 6858 / 7673 with accuracy 89.38

## RESNET50 64batch

Cost at epoch 0 is 1.6738578614492257
Got 6211 / 7673 with accuracy 80.95
Cost at epoch 1 is 0.36827098748360115
Got 6544 / 7673 with accuracy 85.29
Cost at epoch 2 is 0.25158401710303646
Got 6610 / 7673 with accuracy 86.15
Cost at epoch 3 is 0.17947401306054628
Got 6743 / 7673 with accuracy 87.88
Cost at epoch 4 is 0.1432143427306213
Got 6773 / 7673 with accuracy 88.27
Cost at epoch 5 is 0.13597131141040547
Got 6726 / 7673 with accuracy 87.66
Cost at epoch 6 is 0.11176422449016872
Got 6490 / 7673 with accuracy 84.58
Cost at epoch 7 is 0.10507807311995958
Got 6794 / 7673 with accuracy 88.54
Cost at epoch 8 is 0.09668087263148314
Got 6698 / 7673 with accuracy 87.29
Cost at epoch 9 is 0.08156323406393103
Got 6752 / 7673 with accuracy 88.00
Cost at epoch 10 is 0.07199856584954362
Got 6663 / 7673 with accuracy 86.84
Cost at epoch 11 is 0.05500387307424367
Got 6865 / 7673 with accuracy 89.47
Cost at epoch 12 is 0.06714401380192542
Got 6702 / 7673 with accuracy 87.35
Cost at epoch 13 is 0.061244949316460406
Got 6830 / 7673 with accuracy 89.01
Cost at epoch 14 is 0.054343251238326326
Got 6810 / 7673 with accuracy 88.75
Cost at epoch 15 is 0.028602998786062435
Got 6910 / 7673 with accuracy 90.06
Cost at epoch 16 is 0.03473353769662096
Got 6715 / 7673 with accuracy 87.51
Cost at epoch 17 is 0.07053846819186359
Got 6876 / 7673 with accuracy 89.61
Cost at epoch 18 is 0.034487076096691174
Got 6884 / 7673 with accuracy 89.72
Cost at epoch 19 is 0.03011074804117715
Got 6869 / 7673 with accuracy 89.52

In [ ]:
Cost at epoch 0 is 2.5648221334872336
Got 4492 / 7673 with accuracy 58.54
Cost at epoch 1 is 0.491350577893452
Got 6114 / 7673 with accuracy 79.68
Cost at epoch 2 is 0.2690148636734543
Got 6514 / 7673 with accuracy 84.90
Cost at epoch 3 is 0.17898910222216868
Got 6769 / 7673 with accuracy 88.22
Cost at epoch 4 is 0.13287136268515426
Got 6688 / 7673 with accuracy 87.16
Cost at epoch 5 is 0.09830330490009286
Got 6572 / 7673 with accuracy 85.65
Cost at epoch 6 is 0.08001060093206783
Got 6422 / 7673 with accuracy 83.70
Cost at epoch 7 is 0.073534333491877
Got 6796 / 7673 with accuracy 88.57
Cost at epoch 8 is 0.0541052408874608
Got 6557 / 7673 with accuracy 85.46
Cost at epoch 9 is 0.056478030669565596
Got 6713 / 7673 with accuracy 87.49
Cost at epoch 10 is 0.05494157571117215
Got 6632 / 7673 with accuracy 86.43
Cost at epoch 11 is 0.039811522962386686